In [1]:
%pip install ultralytics
import ultralytics


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

# Import libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO



In [5]:
model_best = YOLO("C:/Users/ritik/Desktop/PROJECT/best.pt")

In [43]:
import cv2
import numpy as np
from ultralytics import YOLO
from sort.Sort import Sort

model = YOLO("C:/Users/ritik/Desktop/PROJECT/best.pt")
tracker = Sort(max_age=5, min_hits=2, iou_threshold=0.2)

video_path = 'TPP_1.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties for writer
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object to save output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' or 'XVID' etc.
out = cv2.VideoWriter('output_car_count.mp4', fourcc, fps, (frame_width, frame_height))

counted_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    boxes = results.boxes

    car_detections = []
    if boxes is not None:
        xyxy = boxes.xyxy.cpu().numpy()
        conf = boxes.conf.cpu().numpy()
        cls = boxes.cls.cpu().numpy()

        for box, score, class_id in zip(xyxy, conf, cls):
            if int(class_id) == 2:
                x1, y1, x2, y2 = box
                car_detections.append([x1, y1, x2, y2, score])

    car_detections = np.array(car_detections)
    if len(car_detections) == 0:
        car_detections = np.empty((0, 5))

    tracked_objects = tracker.update(car_detections)

    for x1, y1, x2, y2, track_id in tracked_objects:
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {int(track_id)}", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        if int(track_id) not in counted_ids:
            counted_ids.add(int(track_id))

    cv2.putText(frame, f"Total Cars: {len(counted_ids)}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write the processed frame to output video
    out.write(frame)

    cv2.imshow("Car Counter", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()  # Release the video writer
cv2.destroyAllWindows()



0: 384x640 1 person, 2 cars, 52.2ms
Speed: 6.0ms preprocess, 52.2ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 cars, 45.4ms
Speed: 2.8ms preprocess, 45.4ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 42.7ms
Speed: 3.5ms preprocess, 42.7ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 43.0ms
Speed: 4.0ms preprocess, 43.0ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 42.7ms
Speed: 3.6ms preprocess, 42.7ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 66.6ms
Speed: 5.7ms preprocess, 66.6ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 potted plant, 43.0ms
Speed: 5.5ms preprocess, 43.0ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 truck, 1 potted plant, 

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from sort.Sort import Sort

model = YOLO("C:\Users\ritik\Desktop\New folder\PROJECT\models\Detection.pt")
tracker = Sort(max_age=5, min_hits=2, iou_threshold=0.2)

video_path = 'C:\Users\ritik\Desktop\New folder\PROJECT\TPP\TPP_train\images\64.jpg'
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_car_count.mp4', fourcc, fps, (frame_width, frame_height))

# Define the counting line (horizontal line at y=2/3 of the frame)
line_y = int(frame_height * 2 / 3)
offset = 10  # Tolerance for crossing

counted_ids = set()
track_memory = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    boxes = results.boxes

    car_detections = []
    if boxes is not None:
        xyxy = boxes.xyxy.cpu().numpy()
        conf = boxes.conf.cpu().numpy()
        cls = boxes.cls.cpu().numpy()

        for box, score, class_id in zip(xyxy, conf, cls):
            if int(class_id) == 2:  # Class 2 is 'car' in COCO
                x1, y1, x2, y2 = box
                car_detections.append([x1, y1, x2, y2, score])

    car_detections = np.array(car_detections)
    if len(car_detections) == 0:
        car_detections = np.empty((0, 5))

    tracked_objects = tracker.update(car_detections)

    # Draw counting line
    cv2.line(frame, (0, line_y), (frame_width, line_y), (255, 0, 0), 2)

    for x1, y1, x2, y2, track_id in tracked_objects:
        track_id = int(track_id)
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        # Draw bounding box and ID
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

        # Track the previous position of each ID
        prev_cy = track_memory.get(track_id, None)
        track_memory[track_id] = cy

        # Count only if the car crosses the line from above to below
        if prev_cy is not None and prev_cy < line_y <= cy and track_id not in counted_ids:
            counted_ids.add(track_id)

    cv2.putText(frame, f"Cars Counted: {len(counted_ids)}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(frame)
    cv2.imshow("Car Counter", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

: 